In [148]:
# ======================================================================
# Setup
# ======================================================================
# (Optional in einer eigenen Zelle lassen)
# !pip install pandas numpy plotly

import numpy as np
import pandas as pd
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go

DATA_DIR = Path("data")
METEOSWISS_META_DIR   = DATA_DIR / "meteoswiss_nbcn_meta"
METEOSWISS_YEARLY_DIR = DATA_DIR / "meteoswiss_nbcn_yearly"


In [149]:
# ======================================================================
# Daten einlesen: Schweizer Jahresmittel-Temperatur
# ======================================================================

swiss_mean_path = DATA_DIR / "climate-data-swissmean_regSwiss_1.4.txt"

# Header geht bis Zeile 14, ab Zeile 15 beginnt die Tabelle mit Spalte "time"
ch = pd.read_csv(
    swiss_mean_path,
    sep="\t",
    skiprows=15,
)

# Nur Jahresmittel verwenden
ch = ch[["time", "year"]].rename(columns={"time": "year_int", "year": "temp_mean"})
ch["year_int"] = ch["year_int"].astype(int)

# Nur vollständige Jahre (Temp != NaN)
ch = ch.dropna(subset=["temp_mean"]).reset_index(drop=True)


In [150]:
# ======================================================================
# Anomalien + Trend für Chart 1
# ======================================================================

# Referenz: 1961–1990 (klassische WMO-Klimanormalperiode)
baseline_mask = (ch["year_int"] >= 1961) & (ch["year_int"] <= 1990)
baseline_mean = ch.loc[baseline_mask, "temp_mean"].mean()

ch["anomaly"] = ch["temp_mean"] - baseline_mean

# Glättung: 11-Jahres gleitender Mittelwert
ch["anomaly_smooth"] = (
    ch["anomaly"]
    .rolling(window=11, center=True, min_periods=5)
    .mean()
)

# «Seit Messbeginn +X°C»: Mittel der ersten 30 Jahre vs. letzte 10 Jahre
early_mask = ch["year_int"].between(ch["year_int"].min(), ch["year_int"].min() + 29)
recent_mask = ch["year_int"].between(ch["year_int"].max() - 9, ch["year_int"].max())

warming_since_start = (
    ch.loc[recent_mask, "temp_mean"].mean()
    - ch.loc[early_mask, "temp_mean"].mean()
)

latest_year = int(ch["year_int"].max())
latest_anom = ch.loc[ch["year_int"] == latest_year, "anomaly"].iloc[0]


In [151]:
# ======================================================================
# Chart 1 – Interaktives Plotly-Chart
# ======================================================================

extreme_years = [2003, 2015, 2022]
ext_df = ch[ch["year_int"].isin(extreme_years)]

fig1 = go.Figure()

# dünne Linie: Jahreswerte
fig1.add_trace(
    go.Scatter(
        x=ch["year_int"],
        y=ch["anomaly"],
        mode="lines",
        name="Jahreswerte",
        line=dict(width=1.5),
        hovertemplate="Jahr %{x}<br>Anomalie %{y:.2f} °C<extra></extra>",
    )
)

# dicke Linie: geglätteter Trend
fig1.add_trace(
    go.Scatter(
        x=ch["year_int"],
        y=ch["anomaly_smooth"],
        mode="lines",
        name="11-Jahres-Mittel",
        line=dict(width=4),
        hovertemplate="Jahr %{x}<br>Trend %{y:.2f} °C<extra></extra>",
    )
)

# Marker + Annotationen für Extremjahre
fig1.add_trace(
    go.Scatter(
        x=ext_df["year_int"],
        y=ext_df["anomaly"],
        mode="markers+text",
        name="Extremjahre",
        marker=dict(size=9, symbol="circle"),
        text=[str(y) for y in ext_df["year_int"]],
        textposition="top center",
        hovertemplate="Jahr %{x}<br>Anomalie %{y:.2f} °C<extra></extra>",
    )
)

fig1.update_layout(
    title=(
        f"Wie stark hat sich die Schweiz erwärmt?<br>"
        f"<span style='font-size:0.8em'>"
        f"Seit Messbeginn ≈ +{warming_since_start:.1f} °C "
        f"(Referenz 1961–1990, letzter voller Jahrgang {latest_year})"
        f"</span>"
    ),
    xaxis_title="Jahr",
    yaxis_title="Temperatur-Anomalie (°C, Referenz 1961–1990)",
    template="plotly_white",
    hovermode="x unified",
)

fig1.show()


In [152]:
# ======================================================================
# Metadaten (Stationsliste & Inventar)
# ======================================================================
DATA_DIR = Path("data")
METEOSWISS_DIR = DATA_DIR / "meteoswiss_nbcn_meta"   # Subfolder

stations = pd.read_csv(
    METEOSWISS_DIR / "ogd-nbcn_meta_stations.csv",
    sep=";",
    encoding="cp1252",
)

inventory = pd.read_csv(
    METEOSWISS_DIR / "ogd-nbcn_meta_datainventory.csv",
    sep=";",
    encoding="cp1252",
)

parameters = pd.read_csv(
    METEOSWISS_DIR / "ogd-nbcn_meta_parameters.csv",
    sep=";",
    encoding="cp1252",
)

# Parameter-ID für homogenes Jahresmittel Temperatur
param_temp_year = "ths200y0"
parameters.loc[parameters["parameter_shortname"] == param_temp_year]


,parameter_shortname,parameter_description_de,parameter_description_fr,parameter_description_it,parameter_description_en,parameter_group_de,parameter_group_fr,parameter_group_it,parameter_group_en,parameter_granularity,parameter_decimals,parameter_datatype,parameter_unit
49,ths200y0,Lufttemperatur 2 m über Boden; homogenes Jahre...,Température de l'air à 2 m du sol; moyenne ann...,Temperatura dell'aria a 2 m dal suolo; media a...,Air temperature 2 m above ground; homogeneous ...,Temperatur,Température,Temperatura,Temperature,Y,1,Float,°C


In [153]:
# ======================================================================
# Hilfsfunktion: eine Stationsdatei lesen und Trend berechnen
# ======================================================================

NBCN_YEARLY_DIR = DATA_DIR / "ogd-nbcn_yearly"

def load_station_series(path, parameter=param_temp_year):
    """
    Lies eine ogd-nbcn_*_y.csv Datei
    und gib ein DataFrame mit Spalten ['year', 'value'] zurück.
    """
    df = pd.read_csv(path, sep=";", encoding="cp1252")

    # Zeitspalte suchen (heißt meist 'time' oder ähnlich)
    time_col_candidates = [c for c in df.columns if c.lower().startswith("time")]
    if not time_col_candidates:
        raise ValueError(f"Keine Zeitspalte in {path} gefunden")
    time_col = time_col_candidates[0]

    if parameter not in df.columns:
        # Station misst diesen Parameter evtl. nicht
        return None

    out = df[[time_col, parameter]].copy()
    out = out.rename(columns={time_col: "time", parameter: "value"})

    # Jahreszahl extrahieren
    out["time"] = pd.to_datetime(out["time"], format="%d.%m.%Y %H:%M", errors="coerce")
    out = out.dropna(subset=["time", "value"])
    out["year"] = out["time"].dt.year

    return out[["year", "value"]]


In [154]:
DATA_DIR = Path("data")
METEOSWISS_YEARLY_DIR = DATA_DIR / "meteoswiss_nbcn_yearly"
param_temp_year = "ths200y0"   # homogenes Jahresmittel Temperatur

def load_station_series(path: Path, parameter: str = param_temp_year) -> pd.DataFrame | None:
    df = pd.read_csv(path, sep=";", encoding="cp1252")

    # Parameter vorhanden?
    if parameter not in df.columns:
        return None

    # Jahr aus reference_timestamp holen
    if "reference_timestamp" in df.columns:
        t = pd.to_datetime(df["reference_timestamp"], errors="coerce")
        year = t.dt.year
    elif "year" in df.columns:
        year = df["year"]
    else:
        # Fallback – wird hier eigentlich nicht gebraucht
        return None

    out = pd.DataFrame(
        {
            "year": year,
            "value": df[parameter],
        }
    )

    out = out.dropna(subset=["year", "value"])
    if out.empty:
        return None

    out["year"] = out["year"].astype(int)
    return out


# ----------------------------------------------------------------------
# Trends pro Station berechnen (1961–2024)
# ----------------------------------------------------------------------

csv_files = sorted(METEOSWISS_YEARLY_DIR.glob("ogd-nbcn_*_y.csv"))
print(f"{len(csv_files)} Jahresdateien gefunden")

trend_rows = []

for csv_path in csv_files:
    # Station-Kürzel aus Dateinamen: ogd-nbcn_alt_y.csv -> ALT
    try:
        station_abbr = csv_path.stem.split("_")[1].upper()
    except Exception:
        continue

    df_station = load_station_series(csv_path)
    if df_station is None or df_station.empty:
        continue

    mask = (df_station["year"] >= 1961) & (df_station["year"] <= 2024)
    df_period = df_station.loc[mask].dropna(subset=["value"])

    # Mindestmenge an Jahren
    if df_period["year"].nunique() < 20:
        continue

    x = df_period["year"].values.astype(float)
    y = df_period["value"].values.astype(float)

    slope, intercept = np.polyfit(x, y, 1)
    trend_per_decade = slope * 10.0  # °C pro Dekade

    trend_rows.append(
        {
            "station_abbr": station_abbr,
            "trend_degC_per_decade": trend_per_decade,
            "n_years": df_period["year"].nunique(),
        }
    )

trend_df = pd.DataFrame(trend_rows)
print("trend_df shape:", trend_df.shape)
trend_df.head()

29 Jahresdateien gefunden
trend_df shape: (29, 3)


,station_abbr,trend_degC_per_decade,n_years
0,ALT,0.403915,64
1,ANT,0.391232,64
2,BAS,0.448420,64
3,BER,0.422550,64
4,CDF,0.444986,64


In [155]:
# ======================================================================
# Stationskoordinaten & Höhen dazunehmen
# ======================================================================

station_cols = [
    "station_abbr",
    "station_name",
    "station_canton",
    "station_height_masl",
    "station_coordinates_wgs84_lat",
    "station_coordinates_wgs84_lon",
]

trend_df = trend_df.merge(
    stations[station_cols],
    on="station_abbr",
    how="left",
)

# Einfache Regionseinteilung Nord / Süd / Alpenraum
def classify_region(row):
    h = row["station_height_masl"]
    canton = row["station_canton"]

    if h >= 1000:
        return "Alpenraum"
    # Grobe Einteilung: Südschweiz (TI, GR südliche Täler, VS)
    elif canton in ["TI", "VS", "GR"]:
        return "Süd"
    else:
        return "Nord"

trend_df["region"] = trend_df.apply(classify_region, axis=1)

trend_df.head()


,station_abbr,trend_degC_per_decade,n_years,station_name,station_canton,station_height_masl,station_coordinates_wgs84_lat,station_coordinates_wgs84_lon,region
0,ALT,0.403915,64,Altdorf,UR,438.0,46.887069,8.621894,Nord
1,ANT,0.391232,64,Andermatt,UR,1435.0,46.630914,8.580553,Alpenraum
2,BAS,0.448420,64,Basel / Binningen,BL,316.0,47.541142,7.583525,Nord
3,BER,0.422550,64,Bern / Zollikofen,BE,553.0,46.990744,7.464061,Nord
4,CDF,0.444986,64,La Chaux-de-Fonds,NE,1017.0,47.082947,6.792314,Alpenraum


In [156]:
# ======================================================================
# Chart 2 – Schönes Hover + MapLibre
# ======================================================================

vmin = trend_df["trend_degC_per_decade"].min()
vmax = trend_df["trend_degC_per_decade"].max()

center_lat = trend_df["station_coordinates_wgs84_lat"].mean()
center_lon = trend_df["station_coordinates_wgs84_lon"].mean()

# Schönen Hover bauen
trend_df["hover"] = (
    "<b>" + trend_df["station_name"] + "</b><br>" +
    "Kanton: " + trend_df["station_canton"] + "<br>" +
    "Region: " + trend_df["region"] + "<br>" +
    "Höhe: " + trend_df["station_height_masl"].astype(int).astype(str) + " m ü. M.<br>" +
    "Trend: " + trend_df["trend_degC_per_decade"].round(2).astype(str) + " °C / Dekade<br>" +
    "Lat/Lon: " +
    trend_df["station_coordinates_wgs84_lat"].round(3).astype(str) + "°, " +
    trend_df["station_coordinates_wgs84_lon"].round(3).astype(str) + "°"
)

fig2 = px.scatter_map(
    trend_df,
    lat="station_coordinates_wgs84_lat",
    lon="station_coordinates_wgs84_lon",
    color="trend_degC_per_decade",
    size=np.abs(trend_df["trend_degC_per_decade"]) * 12,
    color_continuous_scale="Turbo",
    range_color=(min(vmin, 0), max(vmax, 0.7)),
    zoom=6,
    hover_name=None,
    hover_data={"hover": True},
    title="Erwärmung pro Dekade seit 1961 an den Swiss-NBCN-Stationen",
    height=650,
)

# Damit Plotly NICHT den dict-Key zeigt (nur unsere HTML):
fig2.update_traces(hovertemplate="%{customdata[0]}<extra></extra>")

fig2.update_layout(
    margin=dict(l=0, r=0, t=60, b=0),
    map={"center": {"lat": center_lat, "lon": center_lon}},
    coloraxis_colorbar=dict(title="Trend<br>°C / Dekade"),
)

fig2.show()
